In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyarabic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import pyarabic.araby as arab
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import string

##Read

In [ ]:
train= pd.read_json("/content/bin_train.json")
dev= pd.read_json("/content/bin_dev.json")
test= pd.read_json("/content/bin_dev.json")

In [ ]:
def preprocess_text(data):
  for i in range(len(data)):
    data.loc[i,'context']=data.loc[i,'context']
    data.loc[i,'sense']=data.loc[i,'sense']
    word=data.loc[i,'word']
    data.loc[i,'context'] = data.loc[i,'context'].replace(word, f"<token>{word}</token>")
    # break
  return data

In [ ]:
train=preprocess_text(train)
dev=preprocess_text(dev)
test=preprocess_text(test)

#Finetune

##camel bert MSA

###prepare data

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB


In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-msa"
tokenizer = BertTokenizer.from_pretrained(model_name)

Loading BERT tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
train_ids = []

# For every sentence...
for sentence, sense in  zip(train['context'],  train['sense']):
  # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    train_sent = tokenizer.encode(
                        sentence, sense,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        max_length = 512,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                        padding=True
                   )
    train_ids.append(train_sent)

print('Original: ', train.loc[0,'context'],  train.loc[0,'sense'])
print('Token IDs:', train_ids[0])
tokenizer.decode(train_ids[0])

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2714: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Original:  <token>تحد</token> تقني عالمي، ما يواجَه من عقبات أو أخطار
Token IDs: [2, 32, 10220, 1066, 2993, 34, 14445, 32, 19, 10220, 1066, 2993, 34, 19816, 8286, 378, 3, 1972, 1, 1908, 6056, 1904, 2075, 2591, 3148, 3]


'[CLS] < token > تحد < / token > تقني عالمي ، [SEP] ما [UNK] من عقبات أو أخطار [SEP]'

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
dev_ids = []

# For every sentence...
for sentence, sense in  zip(dev['context'],  dev['sense']):
  # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    dev_sent = tokenizer.encode(
                        sentence, sense,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        max_length = 512,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                        padding=True
                   )
    dev_ids.append(dev_sent)

print('Original: ', dev.loc[0,'context'],  dev.loc[0,'sense'])
print('Token IDs:', dev_ids[0])
print(tokenizer.decode(dev_ids[0]))

Original:  والجراثيم المسُبّبة <token>للأمراض</token> مختلفة باختلاف المرض، كلّ ما خرج بالكائن الحيّ عن حدِّ الصِّحَّة والاعتدال من عِلّة أو نفاق أو تقصير في أمرٍ .
Token IDs: [2, 3057, 29558, 1, 32, 10220, 1066, 2993, 34, 23748, 2921, 32, 19, 10220, 1066, 2993, 34, 5641, 9817, 2985, 5951, 378, 3, 1, 1972, 5491, 3906, 6565, 1, 1955, 1, 1, 12310, 13164, 1908, 1, 2075, 29890, 2075, 22750, 1912, 1, 18, 3]
[CLS] والجراثيم [UNK] < token > للأمراض < / token > مختلفة باختلاف المرض ، [SEP] [UNK] ما خرج بالكائن [UNK] عن [UNK] [UNK] والاعتدال من [UNK] أو نفاق أو تقصير في [UNK]. [SEP]


In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
test_ids = []

# For every sentence...
for sentence, sense in  zip(test['context'],  test['sense']):
  # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    test_sent = tokenizer.encode(
                        sentence, sense,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        max_length = 512,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                        padding=True
                   )
    test_ids.append(test_sent)

print('Original: ', test.loc[0,'sense'],  test.loc[0,'sense'])
print('Token IDs:', test_ids[0])
print(tokenizer.decode(test_ids[0]))

Original:  كلّ ما خرج بالكائن الحيّ عن حدِّ الصِّحَّة والاعتدال من عِلّة أو نفاق أو تقصير في أمرٍ . كلّ ما خرج بالكائن الحيّ عن حدِّ الصِّحَّة والاعتدال من عِلّة أو نفاق أو تقصير في أمرٍ .
Token IDs: [2, 3057, 29558, 1, 32, 10220, 1066, 2993, 34, 23748, 2921, 32, 19, 10220, 1066, 2993, 34, 5641, 9817, 2985, 5951, 378, 3, 1, 1972, 5491, 3906, 6565, 1, 1955, 1, 1, 12310, 13164, 1908, 1, 2075, 29890, 2075, 22750, 1912, 1, 18, 3]
[CLS] والجراثيم [UNK] < token > للأمراض < / token > مختلفة باختلاف المرض ، [SEP] [UNK] ما خرج بالكائن [UNK] عن [UNK] [UNK] والاعتدال من [UNK] أو نفاق أو تقصير في [UNK]. [SEP]


In [ ]:
print('Max sentence length: ', max([len(sen) for sen in train_ids]))

Max sentence length:  802


In [ ]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
# I've chosen 64 somewhat arbitrarily. It's slightly larger than the
# maximum training sentence length of 47...
MAX_LEN = 512

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long",
                          value=0, truncating="post", padding="post")

dev_ids = pad_sequences(dev_ids, maxlen=MAX_LEN, dtype="long",
                          value=0, truncating="post", padding="post")

test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype="long",
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 512 values...

Padding token: "[PAD]", ID: 0

Done.


In [ ]:
print('Max sentence length: ', max([len(sen) for sen in train_ids]))

Max sentence length:  512


In [ ]:
# Create attention masks
train_masks = []
dev_masks = []
test_masks = []

#For each sentence...
for sent in train_ids:

    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]

    #Store the attention mask for this sentence.
    train_masks.append(att_mask)
# For each sentence...
for sent in dev_ids:

    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]

    # Store the attention mask for this sentence.
    dev_masks.append(att_mask)

# For each sentence...
for sent in test_ids:

    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]

    # Store the attention mask for this sentence.
    test_masks.append(att_mask)

In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype
# for our model.
# train_inputs = torch.tensor(train_inputs)
# dev_inputs = torch.tensor(dev_inputs)

train_inputs=torch.tensor(train_ids)
train_labels = torch.tensor(train['label'])
train_masks = torch.tensor(train_masks)

dev_inputs=torch.tensor(dev_ids)
dev_labels = torch.tensor(dev['label'])
dev_masks = torch.tensor(dev_masks)
dev_index=torch.tensor(dev.index.tolist())

test_inputs=torch.tensor(test_ids)
test_labels = torch.tensor(test['label'])
test_masks = torch.tensor(test_masks)
test_index=torch.tensor(test.index.tolist())




# # Convert to tensors.
# prediction_inputs = torch.tensor(test_ids)
# prediction_masks = torch.tensor(test_masks)
# prediction_labels = torch.tensor(TS_label)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 18

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(dev_inputs, dev_masks, dev_labels, dev_index)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)



# Create the DataLoader.
prediction_data = TensorDataset(test_inputs, test_masks, test_labels, test_index)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


###model

In [ ]:
from transformers import BertForSequenceClassification, BertConfig
#from transformers.optimization import AdamW
from torch.optim import AdamW
# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    model_name, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-msa and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30000, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 3e-5 , # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 10

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

best_validation_loss = np.inf  # Initialize to infinity

model_save_path = f"/content/drive/MyDrive/CAD2025-baseline/CamelBERT/{model_name}_best_model"

# Now, you can use model_save_path as before
print("Model will be saved to:", model_save_path)


Model will be saved to: /content/drive/MyDrive/CAD2025-baseline/CamelBERT/CAMeL-Lab/bert-base-arabic-camelbert-msa_best_model


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score


# # Function to calculate the accuracy of our predictions vs labels
# def flat_accuracy(preds, labels):
#     pred_flat = np.argmax(preds, axis=1).flatten()
#     labels_flat = labels.flatten()
#     acc=np.sum(pred_flat == labels_flat) / len(labels_flat)

#     f1 = f1_score(y_true=labels, y_pred=preds, average=None, labels=[0, 1])
#     precision = precision_score(y_true=labels, y_pred=preds, average=None, labels=[0, 1])
#     recall = recall_score(y_true=labels, y_pred=preds, average=None, labels=[0, 1])

#     return acc, f1, precision, recall


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    acc=np.sum(pred_flat == labels_flat) / len(labels_flat)

    f1 = f1_score(y_true=labels_flat, y_pred=pred_flat, pos_label=1, average='binary', labels=[0, 1])
    precision = precision_score(y_true=labels_flat, y_pred=pred_flat, pos_label=1, average='binary', labels=[0, 1])
    recall = recall_score(y_true=labels_flat, y_pred=pred_flat, pos_label=1, average='binary', labels=[0, 1])
    # print( "f1", f1 , "precision", precision, "recall", recall)

    # # Assuming binary classification and labels are in {0, 1}
    # # Convert labels to the same type as preds for loss calculation
    labels_tensor = torch.tensor(labels_flat, dtype=torch.float)
    preds_tensor = torch.tensor(preds, dtype=torch.float)

    # Calculate loss (ensure your preds and labels are correctly shaped for the loss function)
    # For binary classification, use BCEWithLogitsLoss which expects raw logits as inputs
    loss_fn = torch.nn.BCEWithLogitsLoss()
    loss = loss_fn(preds_tensor[:, 1], labels_tensor)  # Assuming preds are logits and the positive class is at index 1


    return acc, f1, precision, recall, loss

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


###Training

In [ ]:
import time, datetime
# starting time
start = datetime.datetime.now()
# program body starts
#
#


import random
import os

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42
best_validation_loss = np.inf  # Initialize to infinity


random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []
all_logits = []
all_labels = []

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)

        # The call to `model` always returns a tuple, so we need to pull the
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # during evaluation.
    model.eval()

    # Tracking variables
    eval_loss, eval_accuracy, eval_f1, eval_precision, eval_recall  = 0, 0, 0, 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels, b_index = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)
            loss = outputs[0]


        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]


        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        all_logits.extend(logits.tolist())  # Converting to list for easier handling
        # Labels are already numpy arrays from previous steps, just extend the list
        all_labels.extend(label_ids.tolist())  #


        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy, tmp_eval_f1, tmp_eval_precision, tmp_eval_recall, tmp_eval_loss = flat_accuracy(logits, label_ids)

        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        eval_f1 += tmp_eval_f1
        eval_precision += tmp_eval_precision
        eval_recall += tmp_eval_recall
        eval_loss += tmp_eval_loss

        # Track the number of batches
        nb_eval_steps += 1
    # Calculate the average loss over the training data.
    avg_eval_loss = eval_loss / len(validation_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_eval_loss)
    # Check if this is the best model based on validation loss
    if avg_eval_loss < best_validation_loss:
      best_validation_loss = avg_eval_loss  # Update the best validation loss

      # Save this model
      print("Saving model with new best validation loss: {0:.2f}".format(avg_eval_loss))
      model.save_pretrained(model_save_path)
      # Assuming all_logits is a list of lists where each inner list contains logits for an example
      # and all_labels is a simple list of labels
      df_logits = pd.DataFrame(all_logits, columns=[f'Logit_{i}' for i in range(len(all_logits[0]))])
      df_logits['Label'] = all_labels  # Adding a column for the labels


      # Define the file name for the logits and labels CSV
      logits_labels_filename = "logits_and_labels.csv"

      # Construct the full path for the logits and labels CSV file
      logits_labels_path = os.path.join(model_save_path, logits_labels_filename)

      print(logits_labels_path)
      # Save the DataFrame to a CSV file
      df_logits.to_csv(logits_labels_path, index=False)


    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print("  f1: {0:.4f}".format(eval_f1/nb_eval_steps))
    print("  precision: {0:.4f}".format(eval_precision/nb_eval_steps))
    print("  recall: {0:.4f}".format(eval_recall/nb_eval_steps))
    print("  loss: {0:.4f}".format(eval_loss/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")




#
#
# program body ends
time.sleep(1)
# end time
end = datetime.datetime.now()
diff = (end - start)
datetime.timedelta(seconds=10, microseconds=885206)
# total time taken
print(diff)
diff_seconds = int(diff.total_seconds())
minute_seconds, seconds = divmod(diff_seconds, 60)
hours, minutes = divmod(minute_seconds, 60)
# total time taken in h m s
hms = f"{hours}h {minutes}m {seconds}s"
print(hms)


======== Epoch 1 / 10 ========
Training...
  Batch    40  of  6,413.    Elapsed: 0:00:14.
  Batch    80  of  6,413.    Elapsed: 0:00:27.
  Batch   120  of  6,413.    Elapsed: 0:00:40.
  Batch   160  of  6,413.    Elapsed: 0:00:53.
  Batch   200  of  6,413.    Elapsed: 0:01:06.
  Batch   240  of  6,413.    Elapsed: 0:01:20.
  Batch   280  of  6,413.    Elapsed: 0:01:33.
  Batch   320  of  6,413.    Elapsed: 0:01:46.
  Batch   360  of  6,413.    Elapsed: 0:01:59.
  Batch   400  of  6,413.    Elapsed: 0:02:12.
  Batch   440  of  6,413.    Elapsed: 0:02:25.
  Batch   480  of  6,413.    Elapsed: 0:02:38.
  Batch   520  of  6,413.    Elapsed: 0:02:51.
  Batch   560  of  6,413.    Elapsed: 0:03:04.
  Batch   600  of  6,413.    Elapsed: 0:03:17.
  Batch   640  of  6,413.    Elapsed: 0:03:31.
  Batch   680  of  6,413.    Elapsed: 0:03:44.
  Batch   720  of  6,413.    Elapsed: 0:03:57.
  Batch   760  of  6,413.    Elapsed: 0:04:10.
  Batch   800  of  6,413.    Elapsed: 0:04:23.
  Batch   840  o

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py

Saving model with new best validation loss: 0.22
/content/drive/MyDrive/CAD2025-baseline/CamelBERT/CAMeL-Lab/bert-base-arabic-camelbert-msa_best_model/logits_and_labels.csv
  Accuracy: 0.9242
  f1: 0.7380
  precision: 0.7851
  recall: 0.7270
  loss: 0.2196
  Validation took: 0:01:30

======== Epoch 2 / 10 ========
Training...
  Batch    40  of  6,413.    Elapsed: 0:00:13.
  Batch    80  of  6,413.    Elapsed: 0:00:26.
  Batch   120  of  6,413.    Elapsed: 0:00:39.
  Batch   160  of  6,413.    Elapsed: 0:00:52.
  Batch   200  of  6,413.    Elapsed: 0:01:05.
  Batch   240  of  6,413.    Elapsed: 0:01:18.
  Batch   280  of  6,413.    Elapsed: 0:01:32.
  Batch   320  of  6,413.    Elapsed: 0:01:45.
  Batch   360  of  6,413.    Elapsed: 0:01:58.
  Batch   400  of  6,413.    Elapsed: 0:02:11.
  Batch   440  of  6,413.    Elapsed: 0:02:24.
  Batch   480  of  6,413.    Elapsed: 0:02:37.
  Batch   520  of  6,413.    Elapsed: 0:02:50.
  Batch   560  of  6,413.    Elapsed: 0:03:03.
  Batch   600 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py

Saving model with new best validation loss: 0.19
/content/drive/MyDrive/CAD2025-baseline/CamelBERT/CAMeL-Lab/bert-base-arabic-camelbert-msa_best_model/logits_and_labels.csv
  Accuracy: 0.9345
  f1: 0.7854
  precision: 0.7940
  recall: 0.7990
  loss: 0.1906
  Validation took: 0:01:29

======== Epoch 3 / 10 ========
Training...
  Batch    40  of  6,413.    Elapsed: 0:00:13.
  Batch    80  of  6,413.    Elapsed: 0:00:26.
  Batch   120  of  6,413.    Elapsed: 0:00:39.
  Batch   160  of  6,413.    Elapsed: 0:00:52.
  Batch   200  of  6,413.    Elapsed: 0:01:05.
  Batch   240  of  6,413.    Elapsed: 0:01:18.
  Batch   280  of  6,413.    Elapsed: 0:01:32.
  Batch   320  of  6,413.    Elapsed: 0:01:45.
  Batch   360  of  6,413.    Elapsed: 0:01:58.
  Batch   400  of  6,413.    Elapsed: 0:02:11.
  Batch   440  of  6,413.    Elapsed: 0:02:24.
  Batch   480  of  6,413.    Elapsed: 0:02:37.
  Batch   520  of  6,413.    Elapsed: 0:02:50.
  Batch   560  of  6,413.    Elapsed: 0:03:03.
  Batch   600 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classific

Saving model with new best validation loss: 0.18
/content/drive/MyDrive/CAD2025-baseline/CamelBERT/CAMeL-Lab/bert-base-arabic-camelbert-msa_best_model/logits_and_labels.csv
  Accuracy: 0.9417
  f1: 0.8004
  precision: 0.8347
  recall: 0.7890
  loss: 0.1756
  Validation took: 0:01:29

======== Epoch 4 / 10 ========
Training...
  Batch    40  of  6,413.    Elapsed: 0:00:13.
  Batch    80  of  6,413.    Elapsed: 0:00:26.
  Batch   120  of  6,413.    Elapsed: 0:00:39.
  Batch   160  of  6,413.    Elapsed: 0:00:52.
  Batch   200  of  6,413.    Elapsed: 0:01:05.
  Batch   240  of  6,413.    Elapsed: 0:01:18.
  Batch   280  of  6,413.    Elapsed: 0:01:32.
  Batch   320  of  6,413.    Elapsed: 0:01:45.
  Batch   360  of  6,413.    Elapsed: 0:01:58.
  Batch   400  of  6,413.    Elapsed: 0:02:11.
  Batch   440  of  6,413.    Elapsed: 0:02:24.
  Batch   480  of  6,413.    Elapsed: 0:02:37.
  Batch   520  of  6,413.    Elapsed: 0:02:50.
  Batch   560  of  6,413.    Elapsed: 0:03:03.
  Batch   600 

KeyboardInterrupt: 

###Eval

In [ ]:
import time, datetime
# starting time
start = datetime.datetime.now()
# program body starts
#
#


# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))

# Put model in evaluation mode
model.eval()

# Tracking variables
predictions , true_labels = [], []
# Tracking variables
eval_loss, eval_accuracy, eval_f1, eval_precision, eval_recall  = 0, 0, 0, 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# Predict
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels, b_index = batch

  # Telling the model not to compute or store gradients, saving memory and
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  # Calculate the accuracy for this batch of test sentences.
  tmp_eval_accuracy, tmp_eval_f1, tmp_eval_precision, tmp_eval_recall, tmp_eval_loss  = flat_accuracy(logits, label_ids)

  # Accumulate the total accuracy.
  eval_accuracy += tmp_eval_accuracy
  eval_f1 += tmp_eval_f1
  eval_precision += tmp_eval_precision
  eval_recall += tmp_eval_recall
  eval_loss += tmp_eval_loss

  # Track the number of batches
  nb_eval_steps += 1

  # Store predictions and true labels
  all_logits.extend(logits.tolist())  # Converting to list for easier handling
  # Labels are already numpy arrays from previous steps, just extend the list
  all_labels.extend(label_ids.tolist())  #

  df_logits = pd.DataFrame(all_logits, columns=[f'Logit_{i}' for i in range(len(all_logits[0]))])
  df_logits['Label'] = all_labels  # Adding a column for the labels


  # Define the file name for the logits and labels CSV
  logits_labels_filename = "Test_logits_and_labels.csv"

  # Construct the full path for the logits and labels CSV file
  logits_labels_path = os.path.join(model_save_path, logits_labels_filename)

  # print(logits_labels_path)
  # Save the DataFrame to a CSV file
  df_logits.to_csv(logits_labels_path, index=False)

print('    DONE.')
# Report the final accuracy for this validation run.
print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
print("  f1: {0:.4f}".format(eval_f1/nb_eval_steps))
print("  precision: {0:.4f}".format(eval_precision/nb_eval_steps))
print("  recall: {0:.4f}".format(eval_recall/nb_eval_steps))
print("  Validation took: {:}".format(format_time(time.time() - t0)))


#
#
# program body ends
time.sleep(1)
# end time
end = datetime.datetime.now()
diff = (end - start)
datetime.timedelta(seconds=10, microseconds=885206)
# total time taken
print(diff)
diff_seconds = int(diff.total_seconds())
minute_seconds, seconds = divmod(diff_seconds, 60)
hours, minutes = divmod(minute_seconds, 60)
# total time taken in h m s
hms = f"{hours}h {minutes}m {seconds}s"
print(hms)

Predicting labels for 14,551 test sentences...


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-scor

    DONE.
  Accuracy: 0.9524
  f1: 0.8168
  precision: 0.8389
  recall: 0.8102
  Validation took: 0:23:34
0:09:34.684864
0h 9m 34s
